# BCB546X Python Assignment Jupyter Notebook
## Christian Montes-Serey

Loading (importing) the required packages

In [1]:
from Bio import SeqIO
from Bio.Data import CodonTable
import pandas as pd

## First function ##
Dr. X: this gets sequences.
Please finish documenting this function with comments, or in notebook markdown text.
Your descriptions of all functions should contain information about what the function does, as well as information about the return types and arguments.

In [2]:
def get_sequences_from_file(fasta_fn):
    sequence_data_dict = {}
    for record in SeqIO.parse(fasta_fn, "fasta"):
        description = record.description.split()
        species_name = description[1] + " " + description[2]
        sequence_data_dict[species_name] = record.seq
    return(sequence_data_dict)

**First function explained:**  
We are defining (creating) a function called `get_seqences_from_file`. We have to provide the `fasta_fn` argument when using this function. This is, providing a **fasta** format file.  
The `get_seqences_from_file` function does the following:  
1. Creates an empty dictionary object, called `sequence_data_dict`  
2. Starts a `for` loop that goes through each record in the provided **fasta** file (the `SeqIO.parse` function is used to access the file content and the `record` variable is used to store the parsed record). Each loop do:  
2.1. Creates an object called `description`that stores the "fasta header" of the record. The `split()` function separate the header text into a `list` of elements, using the blank space to define each element.  
2.2. Creates an object called `species_name` that have the second and third element of the `description` list object (with a blank space between the elements). This gave us exactly the taxonomic binomial nomenclature for the species whose DNA sequence belongs to.  
2.3. Add the DNA sequence of `record` to our `sequemce_data_dict` dictionary using the species taxonimic name as the index.  
3. After looping through every `record` element of `fasta_fn`, the function uses the `return` function to return the created dictionary.

## Second function ##

In [116]:
def translate_function(string_nucleotides):
    mito_table = CodonTable.unambiguous_dna_by_name["Vertebrate Mitochondrial"]
    aa_seq_string = ""
    i = 0
    while i < len(string_nucleotides):
        codon = string_nucleotides[i:i+3]
        if (codon == 'TAG' or codon == 'TAA' or codon == 'TGA'):
            break
        aa = mito_table.forward_table[codon]
        aa_seq_string += aa
        i += 3
    return(aa_seq_string)

**Second function explained:**  


## Third function ##

In [118]:
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
def imp_translate_function(string_nucleotides):
    coding_dna = Seq(string_nucleotides, IUPAC.unambiguous_dna)
    aa_seq_string = coding_dna.translate()
    return(aa_seq_string)

**Third function explained:**  
Components of this function were taken from [biopython cookbook](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc25)

## Fourth function ##

In [122]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
def compute_molecular_weight(aa_seq):
    analysed_seq = ProteinAnalysis(aa_seq)
    seq_aa_weight = analysed_seq.molecular_weight()
    return(seq_aa_weight)

**Fourth function explained:**  


## Fifth function ##

In [169]:
def get_GC_content(string_nucleotides):
    count = 0
    for base in string_nucleotides:
        if (base == 'G' or base == 'C'):
            count += 1
#            print(count)
    return(count / len(string_nucleotides)*100)

**Fifth function explained:**

# Main #

In [144]:
cytb_seqs = get_sequences_from_file("penguins_cytb.fasta")

In [165]:
penguins_df = pd.read_csv("penguins_mass.csv") # Includes only data for body mass 
species_list = list(penguins_df.species)

In [166]:
penguins_df['molecular_weight'] = 'NaN'
penguins_df['GC_content'] = 'NaN'

In [167]:
penguins_df

,species,mass,molecular_weight,GC_content
0,Aptenodytes forsteri,28.00,NaN,NaN
1,Aptenodytes patagonicus,13.40,NaN,NaN
2,Eudyptes chrysocome,2.80,NaN,NaN
3,Eudyptes chrysolophus,4.50,NaN,NaN
4,Eudyptes sclateri,4.25,NaN,NaN
5,Eudyptula minor,1.60,NaN,NaN
6,Pygoscelis adeliae,4.60,NaN,NaN
7,Pygoscelis antarctica,4.10,NaN,NaN
8,Pygoscelis papua,6.10,NaN,NaN
9,Spheniscus demersus,3.20,NaN,NaN


In [180]:
for key, value in cytb_seqs.items():
    aa_seq = translate_function(value)
    mol_wght = compute_molecular_weight(aa_seq)
    GC_con = get_GC_content(value)
    penguins_df.loc[penguins_df['species'].str.contains(key), 'molecular_weight'] = mol_wght
    penguins_df.loc[penguins_df['species'].str.contains(key), 'GC_content'] = GC_con
    print(key, aa_seq, mol_wght, GC_con)

Aptenodytes forsteri MAPNLRKSHPLLKMINNSLIDLPTPSNISA 3286.865200000001 48.38145231846019
Aptenodytes patagonicus MAPNLRKSHPLLKMINNSLIDLPTPSNISA 3286.865200000001 49.69378827646544
Eudyptes chrysocome MAPNLRKSHPLLKTINNSLIDLPTPSNISA 3256.7731000000013 51.181102362204726
Eudyptes chrysolophus MAPNLRKSHPLLKTINNSLIDLPTPSNISA 3256.7731000000013 50.91863517060368
Eudyptes sclateri MAPNLRKSHPLLKTINNSLIDLPTPSNISA 3256.7731000000013 50.83114610673666
Eudyptula minor MAPNLRKSHPLLKMINNSLIDLPTPSNIST 3316.891200000001 49.256342957130364
Pygoscelis adeliae MAPNLRKSHPLLKMINNSLIDLPTPSNISA 3286.865200000001 49.08136482939632
Pygoscelis antarctica MAPNLRKSHPLLKMINNSLIDLPTPSNISA 3286.865200000001 47.76902887139108
Pygoscelis papua MAPNLRKSHPLLKMINKSLIDLPTPPNISA 3310.9728000000014 47.15660542432196
Spheniscus demersus MAPNLRKSHPLLKTINNSLIDLPTPSNISA 3256.7731000000013 48.29396325459317
Spheniscus humboldti MAPNLRKSHPLLKTINNSLIDLPTPSNISA 3256.7731000000013 49.256342957130364
Spheniscus magellanicus MAPNLRKSHP

C:\Users\Pollo\Anaconda3\lib\site-packages\Bio\Seq.py:152: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


In [181]:
penguins_df

,species,mass,molecular_weight,GC_content
0,Aptenodytes forsteri,28.00,3286.87,48.3815
1,Aptenodytes patagonicus,13.40,3286.87,49.6938
2,Eudyptes chrysocome,2.80,3256.77,51.1811
3,Eudyptes chrysolophus,4.50,3256.77,50.9186
4,Eudyptes sclateri,4.25,3256.77,50.8311
5,Eudyptula minor,1.60,3316.89,49.2563
6,Pygoscelis adeliae,4.60,3286.87,49.0814
7,Pygoscelis antarctica,4.10,3286.87,47.769
8,Pygoscelis papua,6.10,3310.97,47.1566
9,Spheniscus demersus,3.20,3256.77,48.294


In [182]:
import matplotlib.pyplot as plt
import seaborn as sns

In [183]:
%matplotlib inline